In [2]:
!pip install xgboost
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import log_loss
import pandas as pd
import datetime as dt
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from urllib.request import urlretrieve


from sklearn.model_selection import train_test_split

In [12]:
url_train = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_train.csv'
url_test = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_test.csv'
url_test_kaggle = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_test_kaggle.csv'

data_train = pd.read_csv(url_train)
data_test = pd.read_csv(url_test)
data_test_kaggle = pd.read_csv(url_test_kaggle)

In [13]:
# Separamos los set de datos (data_train y data_test) de sus columnas 'Target'
# 'x' son los datos, 'y' es el target:
#   --> x_train son los datos con los que entrenar
#   --> y_train son las soluciones con las que comparar lo entrenado con x_train
#   --> x_test son los datos con los que probar
#   --> y_test son las soluciones con las que comparar lo probado con x_test
x_train = data_train.iloc[:, :-1]
y_train = data_train.iloc[:, -1]
x_test = data_test.iloc[:, :-1]
y_test = data_test.iloc[:, -1]
#Esto lo hizo chechu y esta bien, no cambiar.

In [7]:
!pip install hyperopt
from hyperopt.pyll.base import scope 
import warnings
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials



In [16]:
# Search space definition
# space = hp.uniform('x', -2, 2)
space={'max_depth': hp.quniform("max_depth", 3, 10, 1),
        'gamma': hp.uniform ('gamma', 0.01,8),
        'reg_alpha' : hp.quniform('reg_alpha', 40,50,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.quniform('colsample_bytree', 0.1,1,0.01),
        'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
        'n_estimators': hp.quniform('n_estimators', 30,31,1),
        'learning_rate' : hp.quniform('learning_rate', 0.1, 0.5, 0.01),
        'subsample' : hp.quniform('subsample', 0.5, 1, 0.01),
        'max_features' : hp.quniform('max_features',8,16,1),
        'min_samples_split' :hp.quniform('min_samples_split',2,100,25),
        'eta' : hp.quniform('eta', 0.1,1,0.01),
        'nthread' : hp.quniform('nthread', 1,10,1),

    }
                    
# Regression: 
def hyperparameter_tuning(space):
    warnings.filterwarnings(action='ignore', category=DeprecationWarning)
    model=xgb.XGBRegressor(n_estimators =int(space['n_estimators']), max_depth = int(space['max_depth']), gamma = space['gamma'], reg_lambda=space['reg_lambda'],
                         reg_alpha = int(space['reg_alpha']),min_child_weight=space['min_child_weight'],learning_rate=space['learning_rate'],
                         colsample_bytree=space['colsample_bytree'],max_features=int(space['max_features']),subsample=space['subsample'],
        min_samples_split = int(space['min_samples_split']),eta =space['eta'],nthread =int(space['nthread']),
        objective = 'reg:logistic',booster='gbtree',error='log_loss')
                            
        
    model.fit(x_train, y_train)

    pred = model.predict(x_test)

    logloss = log_loss(y_test, pred)
    print(logloss)
    #change the metric if you like
    return {'loss':logloss, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=1250,
            trials=trials)

print (best)

[18:13:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { error, max_features, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


0.569012225388294                                       
[18:13:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { error, max_features, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


0.5954374263918853                                                               
[18:13:58] WARNING: C:/Users/

In [78]:
#best loss: 0.41740932197098013
best

{'colsample_bytree': 0.17,
 'eta': 0.17,
 'gamma': 2.7833834025261424,
 'learning_rate': 0.5,
 'max_depth': 5.0,
 'max_features': 10.0,
 'min_child_weight': 9.0,
 'min_samples_split': 0.0,
 'n_estimators': 31.0,
 'nthread': 1.0,
 'reg_alpha': 41.0,
 'reg_lambda': 0.7478630636084269,
 'subsample': 0.9500000000000001}

In [11]:
 xg_hyper = xgb.XGBRegressor(n_estimators =30, max_depth =5, gamma = 2.7833834025261424, reg_lambda=0.7478630636084269,
                         reg_alpha = 41,min_child_weight=9,learning_rate=0.5,
                         colsample_bytree= 0.17,subsample=0.9500000000000001,
        eta = 0.17,nthread =1,
        objective = 'reg:logistic',booster='gbtree')
     
xg_hyper.fit(x_train, y_train)

pred_hyper = xg_hyper.predict(x_test)

logloss_hyper = log_loss(y_test, pred_hyper)
logloss_hyper

ValueError: Unknown label type: (0          102.607059
1          104.492157
2        54015.455056
3        47284.786857
4       145213.297297
            ...      
3373      9125.395538
3374    219655.436269
3375      3801.600000
3376       299.632500
3377      1438.125000
Name: Account_Por_TotalAmount, Length: 3378, dtype: float64,)

In [83]:
#Error de train
preds_train_hyper = xg_hyper.predict(x_train)
error_train_hyper = log_loss(y_train, preds_train_hyper)
error_train_hyper

0.25499206135739594

In [84]:
# Sacamos el y_test_de_practica_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

stage_test_de_practica = y_test.to_frame()

oportunidades_test_de_practica['Target'] = stage_test_de_practica
y_test_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
y_test_agrupado_por_opp = y_test_agrupado_por_opp.reset_index()

# Sacamos el preds_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

oportunidades_test_de_practica['Target'] = pred_hyper

preds_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_agrupado_por_opp = preds_agrupado_por_opp.reset_index()

# Calculamos el error con groupby hecho
error = log_loss(y_test_agrupado_por_opp['Target'], np.nan_to_num(preds_agrupado_por_opp['Target'].to_numpy()))
error

0.441051411836789

In [85]:
# Sacamos el preds_test_agrupado_por_opp
preds_test_kaggle_hyper = xg_hyper.predict(data_test_kaggle)

oportunidades_test = data_test_kaggle['Opportunity_ID']
oportunidades_test = oportunidades_test.to_frame()

oportunidades_test['Target'] = preds_test_kaggle_hyper

preds_test_kaggle_agrupado_por_opp = oportunidades_test.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_test_kaggle_agrupado_por_opp = preds_test_kaggle_agrupado_por_opp.reset_index()
preds_test_kaggle_agrupado_por_opp['Opportunity_ID'] = preds_test_kaggle_agrupado_por_opp['Opportunity_ID'].astype(int)

preds_test_kaggle_agrupado_por_opp = preds_test_kaggle_agrupado_por_opp.set_index('Opportunity_ID')
preds_test_kaggle_agrupado_por_opp

,Target
Opportunity_ID,
10689,0.630993
10690,0.539632
10691,0.566063
10692,0.504331
10693,0.662533
...,...
12364,0.787164
12365,0.281665
12366,0.212115


In [86]:
preds_test_kaggle_agrupado_por_opp.to_csv("preds_testxgb_hyper6.csv")


In [17]:
x_train.columns

Index(['ID', 'Region', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Bureaucratic_Code',
       'Account_Created_Date', 'Source', 'Billing_Country', 'Account_Name',
       'Opportunity_ID', 'Account_Owner', 'Opportunity_Owner', 'Account_Type',
       'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Quote_Expiry_Date',
       'Last_Modified_Date', 'Last_Modified_By', 'Product_Family',
       'Product_Name', 'ASP_Currency', 'ASP', 'ASP_(converted_USD)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'TRF', 'Total_Amount_Currency',
       'Total_Amount', 'Total_Taxable_Amount_Currency', 'Total_Taxable_Amount',
       'USD_Total_Amount', 'Ventas_Persona_por_Mes', 'Ventas_Persona_p

In [ ]:
{'colsample_bytree': 0.17,
 'eta': 0.17,
 'gamma': 6.440831951304015,
 'learning_rate': 0.5,
 'max_depth': 6.0,
 'max_features': 10.0,
 'min_child_weight': 6.0,
 'min_samples_split': 0.0,
 'n_estimators': 31.0,
 'nthread': 5.0,
 'reg_alpha': 40.0,
 'reg_lambda': 0.0026168910818640534,
 'subsample': 1.0}

In [ ]:
#error en kaggle: 0.51621
{'colsample_bytree': 0.17,
 'eta': 0.17,
 'gamma': 2.7833834025261424,
 'learning_rate': 0.5,
 'max_depth': 5.0,
 'max_features': 10.0,
 'min_child_weight': 9.0,
 'min_samples_split': 0.0,
 'n_estimators': 31.0,
 'nthread': 1.0,
 'reg_alpha': 41.0,
 'reg_lambda': 0.7478630636084269,
 'subsample': 0.9500000000000001}

In [18]:

#error en kaggle: 0.52
best_params = {'colsample_bytree': 0.15,
 'eta': 0.49,
 'gamma': 7.659001477278592,
 'learning_rate': 0.49,
 'max_depth': 7.0,
 'max_features': 16.0,
 'min_child_weight': 7.0,
 'min_samples_split': 0.0,
 'n_estimators': 30.0,
 'nthread': 10.0,
 'reg_alpha': 41.0,
 'reg_lambda': 0.8295662222146319,
 'subsample': 1.0}

 xg_hyper1 = xgb.XGBRegressor(n_estimators =30, max_depth = 7, gamma =7.659001477278592, reg_lambda=0.8295662222146319,
                         reg_alpha = 41,min_child_weight=7,learning_rate=0.49,
                         colsample_bytree= 0.15,max_features=16,subsample=1,
        eta = 0.49,nthread =10,
        objective = 'reg:logistic',booster='gbtree',error='log_loss')


IndentationError: unexpected indent (<ipython-input-18-660533cb478c>, line 16)